## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,46.0,87,0,11.70,clear sky
1,1,Kapaa,US,22.0752,-159.3190,75.0,100,90,11.50,mist
2,2,Mataura,NZ,-46.1927,168.8643,61.0,76,1,1.01,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,51.8,71,75,31.07,light rain
4,4,Tautira,PF,-17.7333,-149.1500,87.8,66,40,8.05,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
isNum = False
while not isNum:
    try:
        min_temp = float(input("Enter the minimum temperature you're comfortable with: "))
        isNum = True
    except ValueError:
        print("That amount isn't a number. Please input a valid number.")
isNum = False
while not isNum:
    try:
        max_temp = float(input("Enter the maximum temperature you're comfortable with: "))
        isNum = True
    except ValueError:
        print("That amount isn't a number. Please input a valid number.")

Enter the minimum temperature you're comfortable with: 75
Enter the maximum temperature you're comfortable with: 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
comfy_cities_df = city_data_df.loc[((city_data_df["Max Temp"] > min_temp) & (city_data_df["Max Temp"] < max_temp))]
comfy_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Tautira,PF,-17.7333,-149.1500,87.80,66,40,8.05,scattered clouds
14,14,Mandera,KE,3.9366,41.8670,77.25,71,89,12.37,overcast clouds
15,15,Santiago,CL,-33.4569,-70.6483,80.60,36,0,9.22,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,79.05,63,22,2.37,few clouds
18,18,Carnarvon,AU,-24.8667,113.6333,77.00,69,55,17.27,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
comfy_cities_df.count()

City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = comfy_cities_df.dropna()
clean_df.count()

City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tautira,PF,87.80,scattered clouds,-17.7333,-149.1500,
14,Mandera,KE,77.25,overcast clouds,3.9366,41.8670,
15,Santiago,CL,80.60,clear sky,-33.4569,-70.6483,
16,Atuona,PF,79.05,few clouds,-9.8000,-139.0333,
18,Carnarvon,AU,77.00,broken clouds,-24.8667,113.6333,
19,Rikitea,PF,79.72,overcast clouds,-23.1203,-134.9692,
21,Santa Cruz,BO,84.20,broken clouds,-17.8000,-63.1667,
22,Hilo,US,82.40,light rain,19.7297,-155.0900,
26,Vaini,TO,80.60,light rain,-21.2000,-175.2000,
32,Moree,AU,75.20,scattered clouds,-29.4667,149.8500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.count()

City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Lng                    211
Hotel Name             211
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))